In [9]:
import pyarrow.feather as feather
import pandas as pd
from datetime import datetime
import numpy as np
import math
import xgboost

import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt

In [10]:
df_IU = feather.read_feather('df_IU.feather')
df_PQ = feather.read_feather('df_PQ.feather')


df_IU.fillna(0, inplace=True)
df_PQ.fillna(0, inplace=True)

cols = df_PQ.columns

P_sum = sum([df_PQ[name] for name in cols if "P" in name])
Q_sum = sum([df_PQ[name] for name in cols if "Q" in name])

df_PQ.insert(2, "P_sum", P_sum)
df_PQ.insert(3, "Q_sum", Q_sum)
df_PQ["P_sum"].mean()

-1.2239402825693408

In [11]:
# converts a time amount in 'nanoseconds' to an amount in '5 minutes'
def ns_to_5m(x):
    return x/(pow(10,9)*60*5)

In [12]:
# sort values on "DATUM_TIJD"
df_IU = df_IU.sort_values("DATUM_TIJD")
df_PQ = df_PQ.sort_values("DATUM_TIJD")

# .value returns time in nanoseconds, starting form unix time.
# Get starting timestamp and convert this to '5 minutes'
start_time = ns_to_5m(df_IU["DATUM_TIJD"].iloc[1].value)

# Convert each DATETIME timestamp to a float value representing the amount of 5 minutes since start time
df_IU["DATUM_TIJD"] = df_IU['DATUM_TIJD'].apply(lambda x: ns_to_5m(x.value)-start_time)
df_PQ["DATUM_TIJD"] = df_PQ['DATUM_TIJD'].apply(lambda x: ns_to_5m(x.value)-start_time)

df_IU = df_IU.reset_index(drop=True)
df_PQ = df_PQ.reset_index(drop=True)

In [13]:
# Reduce timescale
def weeks_to_5_mins(weeks):
    return 12*24*7*weeks

def reduce_timescale(df, weeks):
    weeks = weeks_to_5_mins(weeks)
    df = df[df["DATUM_TIJD"] <= weeks]
    return df

In [14]:
def calc_percentage_dead_space(Xs):
    dead_space = np.zeros(len(Xs))
    total = Xs[0].stack().value_counts().sum()
    for index, df in enumerate(Xs):
        zeros = df.stack().value_counts()[0]
        dead_space[index] = zeros/total
    return dead_space



# input - df: a Dataframe, chunkSize: the chunk size
# output - a list of DataFrame
# purpose - splits the DataFrame into smaller chunks
def split_dataframe(df, chunk_size): 
    chunks = list()
    num_stations = len(df_IU["STATION"].unique())
    num_chunks = len(df) // (chunk_size*num_stations) + 1
    print(num_chunks)
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks
    

def split_on_stations(df_IU, df_PQ):
    
    df_IU = reduce_timescale(df_IU, 2)
    df_PQ = reduce_timescale(df_PQ, 2)
    
    stations = df_IU["STATION"].unique()

    dfs_IU = []
    dfs_PQ = []

    for station in stations:
        temp_IU = df_IU[df_IU["STATION"]==station]
        temp_PQ = df_PQ[df_PQ["STATION"]==station]

        del temp_PQ["DATUM_TIJD"]
        del temp_IU["STATION"], temp_PQ["STATION"]
        
        temp_PQ.reset_index(drop = True, inplace = True)
        temp_IU.reset_index(drop = True, inplace = True)

        dfs_IU.append(temp_IU)
        dfs_PQ.append(temp_PQ)
    
    return dfs_IU, dfs_PQ

def split_on_weeks(df_IU, df_PQ):   
    dfs_IU = split_dataframe(df_IU, weeks_to_5_mins(2))
    dfs_PQ = split_dataframe(df_PQ, weeks_to_5_mins(2))
    
    for index in range(len(dfs_IU)):
        temp_IU = dfs_IU[index].reset_index(drop=True)
        temp_PQ = dfs_PQ[index].reset_index(drop=True)
        
        del temp_PQ["DATUM_TIJD"]
        del temp_IU["STATION"], temp_PQ["STATION"]
        
        dfs_IU[index] = temp_IU
        dfs_PQ[index] = temp_PQ
    
    return dfs_IU, dfs_PQ
    
    


def cross_validation(Xs, ys):
    matrix = np.zeros((len(ys),len(ys),3))
    alpha = 10
    for x_index, X_train in enumerate(Xs):
        y_train = ys[x_index]
        for y_index, y_test in enumerate(ys):
            print("X: {0}, Y: {1}".format(x_index, y_index))
            X_test = Xs[y_index]
            df_predict = XGboost_regression(X_train, y_train, X_test)
            
            P,_ = predict_sign(df_predict, y_test)
            mse = mean_squared_error(y_test, df_predict)
            R = r2_score(y_test, df_predict)
            
            matrix[x_index, y_index, 0] = P
            matrix[x_index, y_index, 1] = mse
            matrix[x_index, y_index, 2] = R
            
    return matrix

In [15]:
def predict_sign(df_predict, y_test):
    df_sign = df_predict.copy()
    df_y_sign = y_test.copy()
    for col in df_sign.columns:
        df_sign[col] = df_sign[col].apply(lambda x: -1 if x<0 else 1)
        df_y_sign[col] = df_y_sign[col].apply(lambda x: -1 if x<0 else 1)

    df_new_sign = df_sign == df_y_sign
    
    field_accuracies = []
    for col in df_new_sign.columns:
        field_accuracies.append(df_new_sign[col].value_counts(normalize=True).values[0])

    P_accuracies = field_accuracies[::2]
    Q_accuracies = field_accuracies[1::2]
    
    P_avg = sum(P_accuracies)/len(P_accuracies)
    Q_avg = sum(Q_accuracies)/len(Q_accuracies)
    
    return P_avg, Q_avg

In [17]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures


def XGboost_regression(X_train, y_train, X_test):
    model = XGBRegressor(objective='reg:squarederror')
    model.fit(X_train, y_train)

    predict = model.predict(X_test)
    df_predict = pd.DataFrame(predict, columns = y_train.columns, dtype = float)
    return df_predict


In [18]:
Xs, ys = split_on_stations(df_IU, df_PQ)
matrix = cross_validation(Xs, ys)

X: 0, Y: 0
X: 0, Y: 1
X: 0, Y: 2
X: 0, Y: 3
X: 0, Y: 4
X: 0, Y: 5
X: 0, Y: 6
X: 0, Y: 7
X: 0, Y: 8
X: 0, Y: 9
X: 1, Y: 0
X: 1, Y: 1
X: 1, Y: 2
X: 1, Y: 3
X: 1, Y: 4
X: 1, Y: 5
X: 1, Y: 6
X: 1, Y: 7
X: 1, Y: 8
X: 1, Y: 9
X: 2, Y: 0
X: 2, Y: 1
X: 2, Y: 2
X: 2, Y: 3
X: 2, Y: 4
X: 2, Y: 5
X: 2, Y: 6
X: 2, Y: 7
X: 2, Y: 8
X: 2, Y: 9
X: 3, Y: 0
X: 3, Y: 1
X: 3, Y: 2
X: 3, Y: 3
X: 3, Y: 4
X: 3, Y: 5
X: 3, Y: 6
X: 3, Y: 7
X: 3, Y: 8
X: 3, Y: 9
X: 4, Y: 0
X: 4, Y: 1
X: 4, Y: 2
X: 4, Y: 3
X: 4, Y: 4
X: 4, Y: 5
X: 4, Y: 6
X: 4, Y: 7
X: 4, Y: 8
X: 4, Y: 9
X: 5, Y: 0
X: 5, Y: 1
X: 5, Y: 2
X: 5, Y: 3
X: 5, Y: 4
X: 5, Y: 5
X: 5, Y: 6
X: 5, Y: 7
X: 5, Y: 8
X: 5, Y: 9
X: 6, Y: 0
X: 6, Y: 1
X: 6, Y: 2
X: 6, Y: 3
X: 6, Y: 4
X: 6, Y: 5
X: 6, Y: 6
X: 6, Y: 7
X: 6, Y: 8
X: 6, Y: 9
X: 7, Y: 0
X: 7, Y: 1
X: 7, Y: 2
X: 7, Y: 3
X: 7, Y: 4
X: 7, Y: 5
X: 7, Y: 6
X: 7, Y: 7
X: 7, Y: 8
X: 7, Y: 9
X: 8, Y: 0
X: 8, Y: 1
X: 8, Y: 2
X: 8, Y: 3
X: 8, Y: 4
X: 8, Y: 5
X: 8, Y: 6
X: 8, Y: 7
X: 8, Y: 8
X: 8, Y: 9
X: 9, Y: 0

In [30]:
metric = 1
print(matrix[metric])
lst = np.arange(matrix.shape[0])
cols = ["trained_on_" + str(number) for number in lst]
rows = ["tested_on_" + str(number) for number in lst]

pd_crossval_stations = pd.DataFrame(matrix[:,:,metric], index = rows, columns = cols,dtype = float)
pd_crossval_stations

[[ 8.23675828e-01  1.21908558e+01 -4.09548524e+00]
 [ 9.72374847e-01  1.11178445e-02  2.30285751e-01]
 [ 9.25700168e-01  3.30292405e+01 -2.47080405e-01]
 [ 8.75562805e-01  8.85587466e+00 -2.55651688e+02]
 [ 6.37791896e-01  1.10005865e+01 -2.37134937e+00]
 [ 9.22714438e-01  9.22609007e+01 -1.21818330e+00]
 [ 9.72756410e-01  7.12420083e+01 -7.47074820e-01]
 [ 7.97399649e-01  2.86991457e+01 -8.08142654e-01]
 [ 8.80590894e-01  9.77806356e+00 -2.98513055e+03]
 [ 8.36807874e-01  9.67445520e+00 -1.50823183e+02]]


,trained_on_0,trained_on_1,trained_on_2,trained_on_3,trained_on_4,trained_on_5,trained_on_6,trained_on_7,trained_on_8,trained_on_9
tested_on_0,0.000664,4.600247,40.025723,2.419537,2.613171,90.340002,43.860441,15.756260,4.993013,2.835594
tested_on_1,12.190856,0.011118,33.029241,8.855875,11.000587,92.260901,71.242008,28.699146,9.778064,9.674455
tested_on_2,28.140780,19.450609,0.049828,23.766914,21.361541,156.144990,127.340786,18.565284,30.377413,27.972730
tested_on_3,4.229063,5.221819,45.722433,0.000320,4.520874,71.890459,37.934955,25.465763,1.001021,0.967510
tested_on_4,1.272928,4.904698,39.696475,2.042520,0.020187,101.566912,43.620376,12.440288,1.829991,2.702045
tested_on_5,64.740690,37.845778,124.860640,38.741983,60.551983,0.104233,18.829028,192.764756,38.988891,39.736448
tested_on_6,42.894909,23.207294,171.634579,17.115473,30.509354,42.660362,0.049956,80.932785,20.117833,23.552325
tested_on_7,10.220485,5.547874,29.292127,18.917329,11.536856,137.398761,66.655237,0.007112,9.550858,7.775018
tested_on_8,3.598662,5.545384,46.934311,1.135875,4.102117,70.552044,36.961356,23.971730,0.000021,1.208388
tested_on_9,3.829675,5.680774,47.836865,3.032046,4.669203,68.565531,36.035941,28.034935,0.302809,0.006614


In [31]:
temp = Xs[0]

dead_space = calc_percentage_dead_space(Xs)

pd_crossval_stations["dead_space"] = dead_space
pd_crossval_weeks = pd_crossval_stations.reset_index()
pd_crossval_weeks.to_feather("station_crossval_XGboost_mse.feather")

In [70]:
X_train = Xs[0]
y_train = ys[0]
X_test = Xs[2]
X_test = X_test[X_test["DATUM_TIJD"]==0]

model = XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

predict = model.predict(X_test)
df_predict = pd.DataFrame(predict, columns = y_train.columns, dtype = float)



In [69]:
df_predict

,P_sum,Q_sum,P_0,Q_0,P_1,Q_1,P_2,Q_2,P_3,Q_3,...,P_20,Q_20,P_21,Q_21,P_22,Q_22,P_23,Q_23,P_24,Q_24
0,-4.77114,1.963039,0.063502,0.073118,0.414443,-0.186406,0.08755,0.072204,-4.523394,1.378081,...,1.634508e-16,1.634508e-16,1.634508e-16,1.634508e-16,1.634508e-16,1.634508e-16,1.634508e-16,1.634508e-16,1.634508e-16,1.634508e-16
